<a href="https://colab.research.google.com/github/luisosmx/proyect_e2e/blob/main/pof_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
import pandas as pd


In [8]:
URL = 'https://datos.cdmx.gob.mx/dataset/f2046fd5-51b5-4876-b008-bd65d95f9a02/resource/0e8ffe58-28bb-4dde-afcd-e5f5b4de4ccb/download/afluenciastc_simple_01_2023.csv'


In [9]:
  response = requests.get(URL)
  file_path = "afluenciastc_simple_01_2023.csv"
  open(file_path, "wb").write(response.content)

45944517

In [11]:
def data_extraction(url: str) -> pd.DataFrame:

  # Descarga de archivo
  response = requests.get(url)
  file_path = "afluenciastc_simple_01_2023.csv"
  open(file_path, "wb").write(response.content)

  # Leer datos del archivo csv en un DataFrame de Pandas
  df = pd.read_csv("afluenciastc_simple_01_2023.csv", encoding='ISO-8859-1')
  return df 